#Cloud Data Developer - Project Challenge 2021
By John Thomason

## Purpose
To Investigate the benefits of the cloud-native platforms Databricks and Snowflake platforms using the datasets provided by Northwoods Airlines.
To do this, we first load the datasets into Databricks to create reports requested by Northwoods Airlines. Next we load the data from Databricks into Snowflake and create Views in Snowflake to display the data the same as we do in the Databricks reports.

## 1. Load the Data into Databricks

#### Create Airlines Table from Airlines Dataset

In [0]:
%python

airlines_df = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/thom4236@stthomas.edu/airlines.csv", header="true", inferSchema="true")
airlines_df.cache()
display(airlines_df)

IATA_CODE,AIRLINE
UA,United Air Lines Inc.
AA,American Airlines Inc.
US,US Airways Inc.
F9,Frontier Airlines Inc.
B6,JetBlue Airways
OO,Skywest Airlines Inc.
AS,Alaska Airlines Inc.
NK,Spirit Air Lines
WN,Southwest Airlines Co.
DL,Delta Air Lines Inc.


#### Create Airports Table from Airports Dataset

In [0]:
airports_df = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/thom4236@stthomas.edu/airports.csv", header="true", inferSchema="true")
airports_df.cache() # Cache data for faster reuse
display(airports_df)

IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.4404
ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.6819
ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
ACK,Nantucket Memorial Airport,Nantucket,MA,USA,41.25305,-70.06018
ACT,Waco Regional Airport,Waco,TX,USA,31.61129,-97.23052
ACV,Arcata Airport,Arcata/Eureka,CA,USA,40.97812,-124.10862
ACY,Atlantic City International Airport,Atlantic City,NJ,USA,39.45758,-74.57717
ADK,Adak Airport,Adak,AK,USA,51.87796,-176.64603


#### Create Flights Table from Flights Dataset

We have to do some ETL on some of table columns so they are the correct variable type as stated in the requirements.

In [0]:
flights_df = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/thom4236@stthomas.edu/flights/*.csv", header="true", inferSchema="true")
flights_df_fix = flights_df.withColumn("FLIGHT_NUMBER_STR", flights_df['FLIGHT_NUMBER'].cast('string')).drop('FLIGHT_NUMBER').withColumnRenamed('FLIGHT_NUMBER_STR', 'FLIGHT_NUMBER')
flights_df_fix = flights_df_fix.withColumn("SCHEDULED_DEPARTURE_STR", flights_df_fix['SCHEDULED_DEPARTURE'].cast('string')).drop('SCHEDULED_DEPARTURE').withColumnRenamed('SCHEDULED_DEPARTURE_STR', 'SCHEDULED_DEPARTURE')
flights_df_fix = flights_df_fix.withColumn("DEPARTURE_TIME_STR", flights_df_fix['DEPARTURE_TIME'].cast('string')).drop('DEPARTURE_TIME').withColumnRenamed('DEPARTURE_TIME_STR', 'DEPARTURE_TIME')
flights_df_fix = flights_df_fix.withColumn("WHEELS_OFF_STR", flights_df_fix['WHEELS_OFF'].cast('string')).drop('WHEELS_OFF').withColumnRenamed('WHEELS_OFF_STR', 'WHEELS_OFF')
flights_df_fix = flights_df_fix.withColumn("ARRIVAL_TIME_STR", flights_df_fix['ARRIVAL_TIME'].cast('string')).drop('ARRIVAL_TIME').withColumnRenamed('ARRIVAL_TIME_STR', 'ARRIVAL_TIME')
flights_df_fix = flights_df_fix.withColumn("ARRIVAL_DELAY_STR", flights_df_fix['ARRIVAL_DELAY'].cast('string')).drop('ARRIVAL_DELAY').withColumnRenamed('ARRIVAL_DELAY_STR', 'ARRIVAL_DELAY')
flights_df = flights_df_fix
flights_df.cache()
display(flights_df)

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,WHEELS_OFF,ARRIVAL_TIME,ARRIVAL_DELAY
2015,7,1,3,MQ,N935MQ,DFW,CID,16,12,124,119,103,685,1359,4,1352,0,0,null,null,null,null,null,null,3029,1148,1204,1216,1403,11
2015,7,1,3,MQ,N690MQ,DFW,MLI,32,12,122,116,101,691,1414,3,1351,0,0,null,0,0,26,0,0,3283,1149,1221,1233,1417,26
2015,7,1,3,DL,N986AT,TPA,LGA,70,11,161,146,128,1010,1518,7,1430,0,0,null,0,0,6,49,0,884,1149,1259,1310,1525,55
2015,7,1,3,B6,N593JB,SMF,LGB,2,15,80,85,66,387,1312,4,1309,0,0,null,null,null,null,null,null,265,1149,1151,1206,1316,7
2015,7,1,3,B6,N203JB,DCA,TPA,17,15,137,129,112,814,1413,2,1406,0,0,null,null,null,null,null,null,249,1149,1206,1221,1415,9
2015,7,1,3,AS,N477AS,LAS,SEA,0,21,145,146,116,867,1407,9,1415,0,0,null,null,null,null,null,null,603,1150,1150,1211,1416,1
2015,7,1,3,AA,N4YSAA,DFW,BNA,3,21,112,119,93,631,1346,5,1341,0,0,null,null,null,null,null,null,1191,1149,1152,1213,1351,10
2015,7,1,3,NK,N635NK,DEN,LAS,-6,12,110,101,82,628,1216,7,1238,0,0,null,null,null,null,null,null,561,1148,1142,1154,1223,-15
2015,7,1,3,HA,N486HA,HNL,OGG,10,12,39,40,22,100,1232,6,1227,0,0,null,null,null,null,null,null,328,1148,1158,1210,1238,11
2015,7,1,3,MQ,N510MQ,BUF,ORD,-7,23,110,128,74,473,1218,31,1238,0,0,null,null,null,null,null,null,3099,1148,1141,1204,1249,11


## 2. Create Reports in Databricks

### Create views to access the data in the tables

In [0]:
airlines_df.createOrReplaceTempView("airlines")
airports_df.createOrReplaceTempView("airports")
flights_df.createOrReplaceTempView("flights")

### Report 1: Total number of flights by airline and airport on a monthly basis

In [0]:
%sql

SELECT AIRLINE, AIRPORT, MONTH, YEAR, ORIGIN_OR_DESTINATION, SUM(Flight) FLIGHTS from
(select ar.AIRLINE, ap.AIRPORT, f.MONTH, f.YEAR, 'ORIGIN' as ORIGIN_OR_DESTINATION, 1 as Flight from airlines ar, airports ap, flights f where ar.IATA_CODE = f.AIRLINE and ap.IATA_CODE = f.ORIGIN_AIRPORT)
GROUP BY AIRLINE,AIRPORT,MONTH,YEAR,ORIGIN_OR_DESTINATION
UNION ALL
SELECT AIRLINE, AIRPORT, MONTH, YEAR, ORIGIN_OR_DESTINATION, SUM(Flight) FLIGHTS from
(select ar.AIRLINE, ap.AIRPORT, f.MONTH, f.YEAR, 'DESTINATION' as ORIGIN_OR_DESTINATION, 1 as Flight from airlines ar, airports ap, flights f where ar.IATA_CODE = f.AIRLINE and ap.IATA_CODE = f.DESTINATION_AIRPORT)
GROUP BY AIRLINE,AIRPORT,MONTH,YEAR,ORIGIN_OR_DESTINATION
ORDER BY AIRLINE,AIRPORT,MONTH,YEAR,ORIGIN_OR_DESTINATION;

AIRLINE,AIRPORT,MONTH,YEAR,ORIGIN_OR_DESTINATION,FLIGHTS
Alaska Airlines Inc.,Adak Airport,1,2015,DESTINATION,9
Alaska Airlines Inc.,Adak Airport,1,2015,ORIGIN,9
Alaska Airlines Inc.,Adak Airport,2,2015,DESTINATION,8
Alaska Airlines Inc.,Adak Airport,2,2015,ORIGIN,8
Alaska Airlines Inc.,Adak Airport,3,2015,DESTINATION,9
Alaska Airlines Inc.,Adak Airport,3,2015,ORIGIN,9
Alaska Airlines Inc.,Adak Airport,4,2015,DESTINATION,9
Alaska Airlines Inc.,Adak Airport,4,2015,ORIGIN,9
Alaska Airlines Inc.,Adak Airport,5,2015,DESTINATION,9
Alaska Airlines Inc.,Adak Airport,5,2015,ORIGIN,9


### Report 2: On time percentage of each airline for the year 2015

In [0]:
%sql

select o.AIRLINE, CAST(o.On_Time AS DECIMAL) / CAST(t.TOTAL AS DECIMAL) On_Time_Percentage_For_2015  from 
(SELECT DISTINCT AIRLINE, COUNT(AIRLINE) AS On_Time
FROM (select ar.AIRLINE, f.DEPARTURE_DELAY, f.ARRIVAL_DELAY, f.CANCELLED, f.DIVERTED from airlines ar, flights f where ar.IATA_CODE = f.AIRLINE and f.YEAR ='2015')
WHERE (DEPARTURE_DELAY <=0 or DEPARTURE_DELAY is NULL) and
      (ARRIVAL_DELAY <=0 or ARRIVAL_DELAY is NULL) and
      (CANCELLED = 0) and
      (DIVERTED = 0)
GROUP BY AIRLINE) o, 
(SELECT DISTINCT AIRLINE, COUNT(  AIRLINE ) AS Total
FROM (select ar.AIRLINE, f.DEPARTURE_DELAY, f.ARRIVAL_DELAY, f.CANCELLED, f.DIVERTED from airlines ar, flights f where ar.IATA_CODE = f.AIRLINE and f.YEAR ='2015')
GROUP BY AIRLINE) t 
where o.AIRLINE = t.AIRLINE order by AIRLINE

AIRLINE,On_Time_Percentage_For_2015
Alaska Airlines Inc.,0.5914590809
American Airlines Inc.,0.50704666237
American Eagle Airlines Inc.,0.47954794195
Atlantic Southeast Airlines,0.52075711947
Delta Air Lines Inc.,0.5625855186
Frontier Airlines Inc.,0.43259465233
Hawaiian Airlines Inc.,0.54516204063
JetBlue Airways,0.50539779395
Skywest Airlines Inc.,0.54064094621
Southwest Airlines Co.,0.453483488


### Report 3: Airlines with the largest number of delays

In [0]:
%sql

select AIRLINE , sum(air_system_delay) + sum(security_delay) + sum(airline_delay) + sum(late_aircraft_delay) + sum(weather_delay) DELAYS from
(select ar.AIRLINE, 
(CASE 
      WHEN  f.air_system_delay > 0 THEN 1 ELSE 0
END) as air_system_delay, 
(CASE 
      WHEN  f.security_delay > 0 THEN 1 ELSE 0
END) as security_delay, 
(CASE 
      WHEN  f.airline_delay > 0 THEN 1 ELSE 0
END) as airline_delay, 
(CASE 
      WHEN  f.late_aircraft_delay > 0 THEN 1 ELSE 0
END) as late_aircraft_delay,
(CASE 
      WHEN  f.weather_delay > 0 THEN 1 ELSE 0
END) as weather_delay
from airlines ar, flights f where ar.iata_code = f.airline and 
    (not f.air_system_delay is NULL or
     not f.security_delay is NULL or
     not f.airline_delay is NULL or
     not f.late_aircraft_delay is NULL or
     not f.weather_delay is NULL)) group by AIRLINE order by DELAYS desc

AIRLINE,DELAYS
Southwest Airlines Co.,304162
Delta Air Lines Inc.,140225
American Airlines Inc.,135133
United Air Lines Inc.,132316
Atlantic Southeast Airlines,131228
Skywest Airlines Inc.,115166
American Eagle Airlines Inc.,86788
JetBlue Airways,73104
US Airways Inc.,59173
Spirit Air Lines,42544


### Report 4: Airline with the most unique routes

In [0]:
%sql

select airline, sum(Unique_Route) Unique_Routes
from (select distinct ar.airline, 1 as Unique_Route, f.ORIGIN_AIRPORT, f.DESTINATION_AIRPORT from airlines ar, flights f where ar.IATA_CODE = f.AIRLINE) 
group by airline order by sum(Unique_Route) desc
LIMIT 1

airline,Unique_Routes
Atlantic Southeast Airlines,1351


### Report 5: Cancellation reasons by airport

In [0]:
%sql

SELECT AIRPORT,
concat_ws(', ', collect_list(CANCELLATION_REASON)) as CANCELLATION_REASONS 
FROM (select distinct ap.airport,
(CASE WHEN f.CANCELLATION_REASON = 'A' THEN 'Airline/Carrier'
      WHEN f.CANCELLATION_REASON = 'B' THEN 'Weather'
      WHEN f.CANCELLATION_REASON = 'C' THEN 'National Air System'
      WHEN f.CANCELLATION_REASON = 'D' THEN 'Security'
      ELSE NULL
END) as CANCELLATION_REASON 
from airports ap, flights f where ap.iata_code = f.origin_airport and not f.CANCELLATION_REASON is NULL)
  GROUP BY AIRPORT
  order by airport;

AIRPORT,CANCELLATION_REASONS
Aberdeen Regional Airport,"Airline/Carrier, Weather"
Abilene Regional Airport,"Weather, Airline/Carrier"
Abraham Lincoln Capital Airport,"Weather, Airline/Carrier, National Air System"
Adak Airport,Weather
Akron-Canton Regional Airport,"Airline/Carrier, Weather, National Air System"
Albany International Airport,"Airline/Carrier, Weather, National Air System"
Albert J. Ellis Airport,"Airline/Carrier, National Air System, Weather"
Albuquerque International Sunport,"National Air System, Airline/Carrier, Weather"
Alexandria International Airport,"Airline/Carrier, National Air System, Weather"
Alpena County Regional Airport,"Airline/Carrier, Weather, National Air System"


### Report 6: Delay reasons by airport

In [0]:
%sql

select airport, 
CONCAT(air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay) as Delay_Reasons 
from
(select ap.airport, 
       (CASE WHEN sum(f.air_system_delay) > 0 THEN 'Air_system,' ELSE '' END) as air_system_delay,
       (CASE WHEN sum(f.security_delay) > 0 THEN ' Security,' ELSE '' END) as security_delay,
       (CASE WHEN sum(f.airline_delay) > 0 THEN ' Airline,' ELSE '' END) as airline_delay,
       (CASE WHEN sum(f.late_aircraft_delay) > 0 THEN ' Late_aircraft,' ELSE '' END) as late_aircraft_delay,
       (CASE WHEN sum(f.weather_delay) > 0 THEN ' Weather' ELSE '' END) as weather_delay       
from airports ap, flights f where ap.iata_code = f.origin_airport and 
    (not f.air_system_delay is NULL or
     not f.security_delay is NULL or
     not f.airline_delay is NULL or
     not f.late_aircraft_delay is NULL or
     not f.weather_delay is NULL)
group by ap.airport)
order by airport;

airport,Delay_Reasons
Aberdeen Regional Airport,"Air_system, Security, Airline, Late_aircraft, Weather"
Abilene Regional Airport,"Air_system, Security, Airline, Late_aircraft, Weather"
Abraham Lincoln Capital Airport,"Air_system, Security, Airline, Late_aircraft, Weather"
Adak Airport,"Air_system, Security, Airline, Late_aircraft, Weather"
Akron-Canton Regional Airport,"Air_system, Airline, Late_aircraft, Weather"
Albany International Airport,"Air_system, Security, Airline, Late_aircraft, Weather"
Albert J. Ellis Airport,"Air_system, Airline, Late_aircraft,"
Albuquerque International Sunport,"Air_system, Security, Airline, Late_aircraft, Weather"
Alexandria International Airport,"Air_system, Security, Airline, Late_aircraft, Weather"
Alpena County Regional Airport,"Air_system, Airline, Late_aircraft, Weather"


## 3. Load the Databricks Tables in Snowflake

#### Python Snowflake Connection Configuration

In [0]:
options = {
  "sfUrl": "https://es55682.us-central1.gcp.snowflakecomputing.com",
  "sfUser": "",
  "sfPassword": "",
  "sfDatabase": "USER_JOHN",
  "sfSchema": "PUBLIC",
  "sfWarehouse": "INTERVIEW_WH"
}


#### Scala Snowflake Connection Configuration

In [0]:
%scala

val options = Map(
  "sfUrl" -> "https://es55682.us-central1.gcp.snowflakecomputing.com",
  "sfUser" -> "",
  "sfPassword" -> "",
  "sfDatabase" -> "USER_JOHN",
  "sfSchema" -> "PUBLIC",
  "sfWarehouse" -> "INTERVIEW_WH"
)

import net.snowflake.spark.snowflake.Utils

Utils.runQuery(options, """CREATE SCHEMA IF NOT EXISTS PUBLIC""")

options: scala.collection.immutable.Map[String,String] = Map(sfUrl -> https://es55682.us-central1.gcp.snowflakecomputing.com, sfSchema -> PUBLIC, sfPassword -> eE15ib13lr0D8adY, sfUser -> jthom423, sfWarehouse -> INTERVIEW_WH, sfDatabase -> USER_JOHN)
import net.snowflake.spark.snowflake.Utils
res0: java.sql.ResultSet = net.snowflake.client.jdbc.SnowflakeResultSetV1@22f1ed28

#### Delete existing tables in Snowflake

In [0]:
%scala
import net.snowflake.spark.snowflake.Utils
Utils.runQuery(options, """DROP TABLE IF EXISTS airlines""")
Utils.runQuery(options, """DROP TABLE IF EXISTS airports""")
Utils.runQuery(options, """DROP TABLE IF EXISTS flights""")

import net.snowflake.spark.snowflake.Utils
res1: java.sql.ResultSet = net.snowflake.client.jdbc.SnowflakeResultSetV1@4f4ded0f

#### Load Airlines Databricks table into Snowflake

In [0]:
airlines_df.write \
  .format("snowflake") \
  .options(**options) \
  .option("dbtable", "airlines") \
  .save()

In [0]:
df = spark.read \
  .format("snowflake") \
  .options(**options) \
  .option("dbtable", "airlines") \
  .load()

display(df)

IATA_CODE,AIRLINE
UA,United Air Lines Inc.
AA,American Airlines Inc.
US,US Airways Inc.
F9,Frontier Airlines Inc.
B6,JetBlue Airways
OO,Skywest Airlines Inc.
AS,Alaska Airlines Inc.
NK,Spirit Air Lines
WN,Southwest Airlines Co.
DL,Delta Air Lines Inc.


#### Load Airports Databricks table into Snowflake

In [0]:
airports_df.write \
  .format("snowflake") \
  .options(**options) \
  .option("dbtable", "airports") \
  .save()

In [0]:
df = spark.read \
  .format("snowflake") \
  .options(**options) \
  .option("dbtable", "airports") \
  .load()

display(df)

IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.4404
ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.6819
ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
ACK,Nantucket Memorial Airport,Nantucket,MA,USA,41.25305,-70.06018
ACT,Waco Regional Airport,Waco,TX,USA,31.61129,-97.23052
ACV,Arcata Airport,Arcata/Eureka,CA,USA,40.97812,-124.10862
ACY,Atlantic City International Airport,Atlantic City,NJ,USA,39.45758,-74.57717
ADK,Adak Airport,Adak,AK,USA,51.87796,-176.64603


#### Load Flights Databricks table into Snowflake

In [0]:
flights_df.write \
  .format("snowflake") \
  .options(**options) \
  .option("dbtable", "flights") \
  .save()

In [0]:
df = spark.read \
  .format("snowflake") \
  .options(**options) \
  .option("dbtable", "flights") \
  .load()

display(df)

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,WHEELS_OFF,ARRIVAL_TIME,ARRIVAL_DELAY
2015,6,5,5,EV,N858AS,ATL,SHV,-5,20,105,110,85,551,856,5,901,0,0,null,null,null,null,null,null,5056,816,811,831,901,0
2015,6,5,5,EV,N12996,IAD,EWR,-4,20,74,71,40,212,912,11,930,0,0,null,null,null,null,null,null,6176,816,812,832,923,-7
2015,6,5,5,HA,N479HA,OGG,HNL,6,6,35,38,21,100,849,11,851,0,0,null,null,null,null,null,null,155,816,822,828,900,9
2015,6,5,5,AA,N002AA,ATL,MIA,-6,13,119,125,84,594,937,28,1005,0,0,null,null,null,null,null,null,1269,806,800,813,1005,0
2015,6,5,5,OO,N755SK,ORD,DTW,150,19,82,null,null,235,1519,10,1038,1,0,null,null,null,null,null,null,5218,816,1046,1105,1529,null
2015,6,5,5,DL,N662DN,ATL,PNS,-1,12,65,60,43,271,811,5,822,0,0,null,null,null,null,null,null,2212,817,816,828,816,-6
2015,6,5,5,EV,N14977,LGA,CLE,-8,27,109,93,61,419,937,5,1006,0,0,null,null,null,null,null,null,4120,817,809,836,942,-24
2015,6,5,5,UA,N841UA,EWR,ATL,0,25,146,134,101,746,1023,8,1043,0,0,null,null,null,null,null,null,475,817,817,842,1031,-12
2015,6,5,5,UA,N36472,IAD,MCO,1,15,132,117,95,758,1008,7,1029,0,0,null,null,null,null,null,null,1909,817,818,833,1015,-14
2015,6,5,5,B6,N334JB,BOS,JAX,-8,11,169,183,140,1010,1041,32,1107,0,0,null,null,null,null,null,null,1109,818,810,821,1113,6


## Now we go to Snowflake to see the created Tables and Views of these Datasets